# 第一课课后练习

---

在本课程中，我们尝试将数据使用OpenAI的Embedding模型先转化为向量，然后存入Faiss数据库中，最后执行检索，以达到满足从向量数据库执行向量相似度查找的要求。<br/>
注：调用OpenAI的API需要自己申请API Key，详情请参考[OpenAI官网](https://platform.openai.com/docs/quickstart)。

In [1]:
from openai import OpenAI
import numpy as np
client = OpenAI()

# 初始信息
info = ["我最喜欢看全球最大的全性交友网站: Github!", "我在学习如何使用OpenAI的API", "我是项目的负责人"]

# 使用模型嵌入
vector_info = client.embeddings.create(
  model="text-embedding-3-small",
  input=info,
  encoding_format="float"
)

vector_info = vector_info.data
print(f"输入向量化后的结果是:{vector_info}")

输入向量化后的结果是:[Embedding(embedding=[0.014969707, -0.027658092, -0.010118958, 0.07239082, 0.036124855, -0.042498443, -0.04285123, 0.040052492, -0.045367736, -0.021260982, -0.0053123054, 0.005379922, -0.03991138, 0.007120312, 0.044638652, -0.00831977, -0.066511124, -0.024459537, -0.003815923, 0.0047919527, -0.012312084, 0.0038100432, 0.03485484, 0.027422903, 0.014075993, -0.032996856, -0.020614214, -0.0074025374, 0.015945736, -0.03078609, -0.021907749, -0.018262336, 0.010477618, 0.037841726, -0.0017139314, 0.03344371, 0.0076200864, 0.008302131, 0.032620557, -0.0062442375, 0.026458634, -0.0013839335, 0.011653558, 0.0060737263, -0.026294002, -0.0052535087, -0.09586257, 0.020449584, 0.0228485, -0.020202637, -0.060349204, -0.025423806, 0.009219363, -0.0109421145, -0.04122843, 0.022813221, -4.69457e-05, 0.044497542, -0.050000936, 0.03913526, 0.024906393, 0.0100190025, -0.021343298, 0.006796929, 0.027775684, -0.04104028, -0.023730455, -0.06152514, 0.036618747, 0.020978756, -0.008119861, 0.0251651

In [2]:
# 提取出回复的向量信息
total_info = []
for i in range(len(vector_info)):
    total_info.append(vector_info[i].embedding)

total_info = np.array(total_info)
print(np.shape(total_info))

(3, 1536)


In [3]:
# 创建索引
import faiss
dim = 1536  # OpenAI Emebedding模型默认输出的向量维度
index = faiss.IndexFlatIP(dim)
index.add(total_info)
print("已成功创建索引")

已成功创建索引


In [4]:
# 保存数据库
faiss.write_index(index, "First Class Practice.faiss")
print("已成功保存数据库")

已成功保存数据库


In [5]:
# 加载数据库
index = faiss.read_index("First Class Practice.faiss")
print("已成功加载数据库")

已成功加载数据库


In [6]:
# 查询相似向量
user_query = "我喜欢做什么？"

# 使用嵌入模型转化为向量形式
vector_query = client.embeddings.create(
  model="text-embedding-3-small",
  input=user_query,
  encoding_format="float"
)

# 转化为ndarray形式
vector_query = vector_query.data[0].embedding
vector_query = np.array([vector_query])

In [7]:
# 执行查询操作
D, I = index.search(vector_query, k=1)
info_index = I[0][0]
print(f"最相似的向量索引是:{info_index}")
print(f"对应的文本信息是:{info[info_index]}")

最相似的向量索引是:0
对应的文本信息是:我最喜欢看全球最大的全性交友网站: Github!


可以发现，我们已经可以按照文本的相似度关系，从向量数据库中拿到最相似的文本信息了。对于多条数据，此操作是相似的，快去自己动手试试吧✅